In [ ]:


#0704 temp1
#
import cv2
import mediapipe as mp
import speech_recognition as sr
import time
import threading
import queue
import os
from datetime import datetime

# 설정
dorosee_active = False
recording = False
last_audio_check = 0
audio_check_interval = 5  # 음성 인식 주기 (초)
audio_queue = queue.Queue()
audio_thread_running = True

# MediaPipe 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    min_detection_confidence=0.7, 
    min_tracking_confidence=0.5,
    max_num_hands=2
)

# 음성 인식 초기화
recognizer = sr.Recognizer()
mic = sr.Microphone()

# 마이크 초기 설정
print("🎤 마이크 초기화 중...")
with mic as source:
    recognizer.adjust_for_ambient_noise(source, duration=1)
    print("✅ 마이크 초기화 완료")

def audio_recognition_thread():
    """음성 인식을 별도 스레드에서 처리"""
    global dorosee_active, audio_thread_running
    
    while audio_thread_running:
        try:
            if not dorosee_active:
                print("🎤 음성 대기 중... ('도로시'라고 말해주세요)")
                with mic as source:
                    # 배경 소음 조절을 짧게
                    recognizer.adjust_for_ambient_noise(source, duration=0.5)
                    # 음성 듣기 (타임아웃 설정)
                    audio = recognizer.listen(source, timeout=2, phrase_time_limit=4)
                    
                print("음성 인식 중...")
                text = recognizer.recognize_google(audio, language='ko-KR')
                print(f"인식된 텍스트: {text}")
                
                # 인식된 텍스트 파일에 저장
                with open("dorosee.txt", "w", encoding="utf-8") as f:
                    f.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: {text}")
                
                # '도로시' 키워드 검출
                if "도로시" in text.lower():
                    dorosee_active = True
                    print("도로시가 활성화되었습니다! (음성 인식)")
                    
        except sr.WaitTimeoutError:
            # 타임아웃은 정상적인 상황이므로 조용히 처리
            pass
        except sr.UnknownValueError:
            print("❓ 음성을 명확히 인식할 수 없습니다.")
        except sr.RequestError as e:
            print(f"❗ 음성 인식 서비스 오류: {e}")
        except Exception as e:
            print(f"❗ 음성 인식 오류: {e}")
        
        # 잠시 대기
        time.sleep(0.1)

def main():
    global dorosee_active, recording, audio_thread_running
    
    # 음성 인식 스레드 시작
    audio_thread = threading.Thread(target=audio_recognition_thread, daemon=True)
    audio_thread.start()
    
    # 웹캠 초기화
    print("📹 웹캠 초기화 중...")
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("웹캠을 열 수 없습니다.")
        return
    
    # 웹캠 해상도 설정
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    print("✅ 웹캠 초기화 완료")
    print("=" * 50)
    print("도로시 대기 중...")
    print("   - 손을 화면에 보이거나")
    print("   - '도로시'라고 말하면 녹화가 시작됩니다!")
    print("   - 'q' 키로 종료")
    print("=" * 50)
    
    out = None
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("❗ 프레임을 읽을 수 없습니다.")
                break
            
            # 프레임 좌우 반전 (거울 효과)
            frame = cv2.flip(frame, 1)
            
            # 손 인식 처리
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            hand_detected = results.multi_hand_landmarks is not None
            
            # 손이 인식되었을 때
            if hand_detected:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2),
                        mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
                    )
                
                if not dorosee_active:
                    dorosee_active = True
                    print("도로시가 활성화되었습니다! (손 인식)")
            
            # 상태 텍스트 화면에 표시
            if dorosee_active:
                status_text = "DOROSEE ACTIVE - RECORDING"
                color = (0, 255, 0)  # 초록색
            else:
                status_text = "DOROSEE STANDBY"
                color = (0, 0, 255)  # 빨간색
            
            # 텍스트 배경 박스
            text_size = cv2.getTextSize(status_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
            cv2.rectangle(frame, (10, 10), (text_size[0] + 20, 40), (0, 0, 0), -1)
            cv2.putText(frame, status_text, (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            
            # 녹화 시작
            if dorosee_active and not recording:
                print("🎥 영상 녹화를 시작합니다!")
                
                # 타임스탬프로 파일명 생성
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f'dorosee_record_{timestamp}.avi'
                
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                out = cv2.VideoWriter(filename, fourcc, 20.0, (frame.shape[1], frame.shape[0]))
                recording = True
                
                print(f"📁 파일명: {filename}")
            
            # 녹화 중일 때 프레임 저장
            if recording and out is not None:
                out.write(frame)
                
                # 녹화 중 표시
                cv2.circle(frame, (frame.shape[1] - 30, 30), 10, (0, 0, 255), -1)
                cv2.putText(frame, "REC", (frame.shape[1] - 60, 50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            
            # 프레임 출력
            cv2.imshow('Doro_see Webcam', frame)
            
            # 종료 조건 확인
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                print("종료 키 입력됨.")
                break
            elif key == ord('r'):  # 'r' 키로 리셋
                dorosee_active = False
                if recording:
                    print("⏹️ 녹화 중지")
                    recording = False
                    if out is not None:
                        out.release()
                        out = None
                print("🔄 도로시 리셋됨")
            
            # 창이 닫혔는지 확인
            if cv2.getWindowProperty('Doro_see Webcam', cv2.WND_PROP_VISIBLE) < 1:
                print("창이 닫혀서 프로그램을 종료합니다.")
                break
                
    except KeyboardInterrupt:
        print("\n⚠️ 사용자가 프로그램을 중단했습니다.")
    except Exception as e:
        print(f"❗ 오류 발생: {e}")
    finally:
        # 종료 처리
        audio_thread_running = False
        
        if recording and out is not None:
            out.release()
            print("✅ 녹화 파일이 저장되었습니다.")
        
        cap.release()
        hands.close()
        cv2.destroyAllWindows()
        print("프로그램이 정상적으로 종료되었습니다.")

if __name__ == "__main__":
    main()